In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import os
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch  
import numpy as np  
from PIL import Image  # For loading images  
import os

In [3]:
# Define the MoleCNN class
class MoleCNN(nn.Module):
    def __init__(self, num_classes=1, resnet_weights_path=None):  # Add resnet_weights_path argument
        super(MoleCNN, self).__init__()
        
        # Load ResNet50 model without downloading pre-trained weights
        self.resnet = models.resnet50(weights=None)
        
        # Load ResNet50 weights from the local file
        if resnet_weights_path and os.path.exists(resnet_weights_path):
            print(f"Loading ResNet50 weights from: {resnet_weights_path}")
            state_dict = torch.load(resnet_weights_path, map_location=torch.device('cpu'))
            self.resnet.load_state_dict(state_dict)
        else:
            print("No ResNet50 weights provided. Using randomly initialized weights.")
        
        # Freeze all parameters in the pretrained model
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # Unfreeze the last layer (layer4) for fine-tuning
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        # Replace the final fully connected layer
        # ResNet50's final layer has 2048 features
        self.resnet.fc = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        # Get the features from ResNet
        x = self.resnet(x)
        
        # Apply sigmoid for binary classification probability
        probability_of_class_1 = x
        
        return probability_of_class_1

# Path to the ResNet50 weights
resnet_weights_path = '/kaggle/input/testmodel2/pytorch/default/1/resnet50.pth'

# Initialize the MoleCNN model with ResNet50 weights
mole_model = MoleCNN(resnet_weights_path=resnet_weights_path)

# Load the MoleCNN state_dict
mole_model.load_state_dict(torch.load('/kaggle/input/mole_cnn/pytorch/default/1/mole_cnn_state_dict.pth', map_location=torch.device('cpu')))

# Put the model in evaluation mode
mole_model.eval()

print("MoleCNN model loaded successfully from 'mole_cnn_state_dict.pth'")

Loading ResNet50 weights from: /kaggle/input/testmodel2/pytorch/default/1/resnet50.pth


<ipython-input-3-d9e1dcd1a318>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resnet_weights_path, map_location=torch.device('cpu'))
<ipython-inpu

MoleCNN model loaded successfully from 'mole_cnn_state_dict.pth'


In [4]:
import h5py
import os
from PIL import Image
import io

# Paths
hdf5_path = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
output_folder = "/kaggle/working/jpg_test_images"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Open HDF5 file
with h5py.File(hdf5_path, "r") as f:
    for isic_id in f.keys():  # Iterate over image keys
        raw_data = f[isic_id][()]  # Read raw image data

        try:
            image = Image.open(io.BytesIO(raw_data))  # Decode image
            image = image.convert("RGB")  # Ensure RGB format

            # Save as JPG
            save_path = os.path.join(output_folder, f"{isic_id}.jpg")
            image.save(save_path, "JPEG")

            print(f"Saved: {save_path}")
        except Exception as e:
            print(f"Skipping {isic_id}: {e}")


Saved: /kaggle/working/jpg_test_images/ISIC_0015657.jpg
Saved: /kaggle/working/jpg_test_images/ISIC_0015729.jpg
Saved: /kaggle/working/jpg_test_images/ISIC_0015740.jpg


In [5]:
class TestMoleDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, img))]
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")  # Open and convert to RGB

        if self.transform:
            image = self.transform(image)

        return image, img_path  # Return image path to track predictions


In [6]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = TestMoleDataset(image_dir=output_folder, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)


In [7]:
import torch
import pandas as pd

# Load trained model (adjust path if needed)
mole_model.eval()  # Set to evaluation mode

# Ensure model runs on GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mole_model.to(device)

# Store results
results = []

# Disable gradients for inference
with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)
        logits = mole_model(images)  # Model outputs logits

        # Apply sigmoid to get the probability of class 1
        probabilities = torch.sigmoid(logits).squeeze(1)  # Squeeze to remove unnecessary dimensions

        # Store probabilities with image names (excluding the extension)
        for img_path, prob in zip(img_paths, probabilities.cpu().numpy()):
            # Extract the ISIC ID from the image path (e.g., ISIC_0015657.jpg -> ISIC_0015657)
            isic_id = os.path.splitext(os.path.basename(img_path))[0]
            results.append({"isic_id": isic_id, "target": prob})

# Convert results to DataFrame and save
df = pd.DataFrame(results)
df.to_csv("/kaggle/working/submission.csv", index=False)

print("Predictions saved to submission.csv")
df.head()

Predictions saved to submission.csv


,isic_id,target
0,ISIC_0015740,0.110296
1,ISIC_0015657,0.347629
2,ISIC_0015729,0.057784


In [8]:
import os
import pandas as pd
import shutil

file_path = "/kaggle/working/submission.csv"

# Check if the file exists
if os.path.exists(file_path):
    print("✅ Submission file saved successfully!")

    # Display first few rows
    df_check = pd.read_csv(file_path)
    print(df_check.head())
else:
    print("❌ Submission file NOT found. Check the file path.")

# Cleanup: Remove all files and directories except submission.csv
working_dir = "/kaggle/working/"
file_to_keep = "submission.csv"

# List all files and folders in the directory
items = os.listdir(working_dir)

# Iterate through all items
for item in items:
    item_path = os.path.join(working_dir, item)

    # Remove files other than submission.csv
    if os.path.isfile(item_path) and item != file_to_keep:
        os.remove(item_path)
        print(f"Deleted file: {item_path}")

    # Remove any directories
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)
        print(f"Deleted folder: {item_path}")

print("Cleanup complete.")


✅ Submission file saved successfully!
        isic_id    target
0  ISIC_0015740  0.110296
1  ISIC_0015657  0.347629
2  ISIC_0015729  0.057784
Deleted file: /kaggle/working/__notebook__.ipynb
Deleted folder: /kaggle/working/jpg_test_images
Cleanup complete.
